In [1]:
import numpy as np
import pandas as pd
import torch
import torchvision
import torch.nn as nn
from torch.nn.functional import interpolate 
import matplotlib.pyplot as plt

from pathlib import Path

# generation imports
from pippi.soundbuffer import SoundBuffer
from pippi import dsp,fx
import param_generation as pg
import _pickle as pickle
from IPython.display import Audio
from feature_extraction.mir_utils import *
###
import torch.utils.data as utils
import torchvision.transforms as transforms
from PIL import Image
###
import scipy.stats as ss
import common_vars as comv
import imp
import librosa
import librosa.display
import sys
import csv
sys.path.append("..")

import helpers
imp.reload(helpers)
imp.reload(comv)
imp.reload(pg)
from feature_extraction import pytorch_models as tm
imp.reload(tm)
from helpers import *

# from common_vars import SR
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
# save_path="./generated_sample_pack/"
# device = "cpu"
# stack_size=3
# BATCH_SIZE=1
# NUM_BINS=100

# classes=comv.classes
# classes_ranked=comv.classes_ranked
# cDict={v:i for i,v in enumerate(classes)}

# drum_groups=['tom_low','snare','hihat_closed','rim','synth_noise','clap','kick','hihat_open','tom_mid']


# drum_df=pd.DataFrame(columns=["name","fc","cnnlstm","env+freq","consensus","stack_size"])

/home/asalimi/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/home/asalimi/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method

In [2]:
out,params = StackMaker(2)
a = memToAud(out)
Audio(a,rate=44100,autoplay=True)

In [3]:
# transformations to the audio (by default, the SR rate of generations is 44100)
# model is trained on 5512 SR
SCALE_FACTOR = 1/8

signal = interpolate(torch.tensor(a).reshape([1,1,-1]),scale_factor = SCALE_FACTOR,recompute_scale_factor=False).reshape(1,-1)
Audio(signal.reshape([-1]),rate=5512,autoplay=True)

In [33]:
import torchaudio
spec = torchaudio.functional.spectrogram

FREQ_BINS=30
TIME_STEPS=10
class specTrans(object):
    def __init__(self,num_mels=50,sr=SR,time_steps=20,amp_to_power=True):
        self.sr = sr
        self.amp_to_power=amp_to_power
        self.num_mels=num_mels
        self.ampP=torchaudio.transforms.AmplitudeToDB(stype='power',top_db=60)
        self.melP=torchaudio.transforms.MelScale(n_mels=self.num_mels, sample_rate=sr,n_stft=None)
        self.hop_step=time_steps-1
    def __call__(self, sample):
        
        wf=sample
        wf=wf.reshape(-1,len(wf))
        sample_length=self.sr

        num_bins=wf[0].shape[0]
        win_length=self.sr//17
        hop_step=self.sr//self.hop_step
        window=torch.tensor([1]*win_length)
        s=spec(wf, 0, window, num_bins, hop_step, win_length,2,normalized=True)
        s=self.melP(s)
        if self.amp_to_power:
            s=self.ampP(s)
        s = s - s.min()
        s = s/s.abs().max()

        freq=s
        freq[torch.isnan(freq)]=0
        freq=freq
        return freq.detach()
spec_tf = specTrans(FREQ_BINS,time_steps=TIME_STEPS,sr=SR//4,amp_to_power=True)

In [34]:
cnet.conv_1d[9].weight.device

device(type='cuda', index=1)

In [266]:
# load model here
try:
    del cnet
except:
    pass
cnet = tm.Conv_Spec_DVN(embed_only=False,device=device).to(device)
checkpoint = torch.load("feature_extraction/models/1d_conv/1.187_0.2615_.checkpoint")
cnet.load_state_dict(checkpoint['model_state_dict'])
cnet.to(device)
cnet.eval()
pass

In [267]:
spec_signal = interpolate(torch.tensor(a).reshape([1,1,-1]),scale_factor = 0.25,recompute_scale_factor=False).reshape(1,-1)
spectrogram = spec_tf(spec_signal[0].float()).to(device)

In [268]:
# make sound and evaluate
def make_and_eval(stack_size = 1):
  
    out,params = StackMaker(stack_size)
    a = memToAud(out)
    # transformations to the audio (by default, the SR rate of generations is 44100)
    signal = interpolate(torch.tensor(a).reshape([1,1,-1]),scale_factor = SCALE_FACTOR,recompute_scale_factor=False).reshape(1,-1).to(device)
    with torch.no_grad():
        ps = cnet(signal,spectrogram)
    return a,signal,ps.cpu().numpy()[0],torch.sigmoid(ps).cpu().numpy()[0]
a,a_downsampled,ps,pss = make_and_eval()
Audio(a_downsampled.cpu().reshape([-1]),rate=5512,autoplay=True)

In [269]:
Audio(a.reshape([-1]),rate=44100,autoplay=True)

In [291]:
biggest_p = []
for i in range(1000):
    a,a_ds,ps,pss= make_and_eval(2)
    print(pss,end="\r")
    if pss[0]>pss[1]:
        print("found 1")
        break
    biggest_p.append(ps[0])
Audio(a,rate=SR,autoplay=True)

found 1782  0.25363693]


In [192]:
z = np.nan_to_num(np.array(biggest_p),nan = -100)
z.sort()
z

array([], dtype=float64)

In [193]:
m = nn.Conv1d(1, 128, 3 , stride=2)
input = torch.randn(20, 1, 50)
output = m(input)
input.shape,output.shape

(torch.Size([20, 1, 50]), torch.Size([20, 128, 24]))

In [194]:

m = nn.Conv1d(16, 128, 3, stride=2)
input = torch.randn(20, 16, 50)
output = m(input)
input.shape,output.shape

(torch.Size([20, 16, 50]), torch.Size([20, 128, 24]))